In [ ]:
from sentence_transformers import SentenceTransformer
import faiss

sentence_encoder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")  # dim=384
fields = ['title', 'description', 'channel', 'date']  # etc.
field_embeds = sentence_encoder.encode(fields, convert_to_tensor=True)  # shape: [num_fields, 384]


In [ ]:
import torch
import torch.nn as nn
from transformers import BertModel, BertConfig

class MetadataFusion(nn.Module):
    def __init__(self, hidden_dim=384, num_fields=4):
        super().__init__()
        config = BertConfig(
            hidden_size=hidden_dim,
            num_attention_heads=6,
            num_hidden_layers=2,
            intermediate_size=hidden_dim * 4,
            max_position_embeddings=num_fields + 1,  # +1 for [CLS]
            num_labels=1
        )
        self.bert = BertModel(config)
        self.cls_token = nn.Parameter(torch.randn(1, 1, hidden_dim))  # [CLS] token
        
        # Positional embeddings to help identify which field is which
        self.field_type_embed = nn.Embedding(num_fields, hidden_dim)
        self.num_fields = num_fields

    def forward(self, field_embeddings):
        """
        field_embeddings: Tensor of shape [batch_size, num_fields, hidden_dim]
        """
        batch_size = field_embeddings.size(0)
        device = field_embeddings.device
        
        # Add learned field-type embeddings (field identity)
        field_positions = torch.arange(self.num_fields, device=device)
        field_type_bias = self.field_type_embed(field_positions)  # [num_fields, hidden_dim]
        field_type_bias = field_type_bias.unsqueeze(0).expand(batch_size, -1, -1)  # [batch, num_fields, hidden_dim]
        enriched_fields = field_embeddings + field_type_bias
        
        # Prepend [CLS] token
        cls_tokens = self.cls_token.expand(batch_size, -1, -1)  # shape: [batch_size, 1, hidden_dim]
        tokens = torch.cat([cls_tokens, field_embeddings], dim=1)  # [batch, num_fields + 1, dim]

        attention_mask = torch.ones(tokens.shape[:2], dtype=torch.long).to(tokens.device)
        output = self.bert(inputs_embeds=tokens, attention_mask=attention_mask)
        fused = output.last_hidden_state[:, 0]  # take [CLS] token
        return fused  # shape: [batch_size, hidden_dim]


In [ ]:
# Example: batch of 32 videos, each with 5 fields
field_embeds = torch.stack([
    sentence_encoder.encode([title, desc, channel, date, tags], convert_to_tensor=True)
    for (title, desc, channel, date, tags) in batch
])  # shape: [32, 5, 384]

fuser = MetadataFusion(hidden_dim=384, num_fields=5).cuda()
fused_repr = fuser(field_embeds.cuda())  # [32, 384]


NameError: name 'batch' is not defined